#Pipelining

we have explored the performaces of various models and resampling. After examinination we have decided extra trees classifiers resampled using SMOTE works best for this dataset. Now we run the module and pipeline the delayed aircraft rows to module 3.

In [0]:
import pandas as pd
import json
from pandas.io.json import json_normalize  
from google.colab import drive
import datetime
drive.mount('/content/gdrive/',force_remount = True)

Mounted at /content/gdrive/


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [0]:
df = pd.read_csv("gdrive/My Drive/Colab Notebooks/merged.csv")

In [0]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes==object# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()

# import labelencoder
from sklearn.preprocessing import LabelEncoder# instantiate labelencoder object
le = LabelEncoder()

# apply le on categorical feature columns
df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))

In [0]:
y = np.array(df['ArrDel15'])
ry = np.array(df[['ArrTime','ArrDelayMinutes','CRSArrTime']])
df = df.drop(['Unnamed: 0','ArrDel15','ArrTime','ArrDelayMinutes','CRSArrTime','round_Atime','round_Dtime', 'DepTime','A_date','itime_A','itime_D','FlightDate'],axis=1)
X = np.array(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

df.columns

Index(['windspeedKmph_A', 'DewPointC_A', 'cloudcover_A', 'precipMM_A',
       'pressure_A', 'WindGustKmph_A', 'visibility_A', 'tempC_A',
       'WindChillC_A', 'winddirDegree_A', 'humidity_A', 'date_A', 'airport_A',
       'windspeedKmph_D', 'DewPointC_D', 'cloudcover_D', 'precipMM_D',
       'pressure_D', 'WindGustKmph_D', 'visibility_D', 'tempC_D',
       'WindChillC_D', 'winddirDegree_D', 'humidity_D', 'date_D', 'airport_D',
       'Year', 'Quarter', 'Month', 'DayofMonth', 'Origin', 'Dest',
       'CRSDepTime', 'DepDelayMinutes', 'DepDel15'],
      dtype='object')

In [0]:
print(y_test)

[0. 1. 1. ... 0. 0. 1.]


In [0]:
from imblearn.over_sampling import SMOTE # SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf = clf.fit(X_sm, y_sm)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94    281959
         1.0       0.77      0.77      0.77     74219

    accuracy                           0.90    356178
   macro avg       0.85      0.85      0.85    356178
weighted avg       0.90      0.90      0.90    356178



In [0]:
x_df = pd.DataFrame(data = X_test, columns = ['windspeedKmph_A', 'DewPointC_A', 'cloudcover_A', 'precipMM_A',
       'pressure_A', 'WindGustKmph_A', 'visibility_A', 'tempC_A',
       'WindChillC_A', 'winddirDegree_A', 'humidity_A', 'date_A', 'airport_A',
       'windspeedKmph_D', 'DewPointC_D', 'cloudcover_D', 'precipMM_D',
       'pressure_D', 'WindGustKmph_D', 'visibility_D', 'tempC_D',
       'WindChillC_D', 'winddirDegree_D', 'humidity_D', 'date_D', 'airport_D',
       'Year', 'Quarter', 'Month', 'DayofMonth', 'Origin', 'Dest',
       'CRSDepTime', 'DepDelayMinutes', 'DepDel15'])

ry_df = pd.DataFrame(data = ry, columns = ['ArrTime','ArrDelayMinutes','CRSArrTime'])
pred_df = pd.DataFrame(data = predictions, columns = ['predicted'])
pred_df = pred_df.merge(x_df, left_index=True, right_index=True)
pred_df = pred_df.merge(ry_df, left_index=True, right_index=True)

In [0]:
pred_df = pred_df[pred_df.predicted != 0.0]

In [0]:
pred_df.head()

,predicted,windspeedKmph_A,DewPointC_A,cloudcover_A,precipMM_A,pressure_A,WindGustKmph_A,visibility_A,tempC_A,WindChillC_A,winddirDegree_A,humidity_A,date_A,airport_A,windspeedKmph_D,DewPointC_D,cloudcover_D,precipMM_D,pressure_D,WindGustKmph_D,visibility_D,tempC_D,WindChillC_D,winddirDegree_D,humidity_D,date_D,airport_D,Year,Quarter,Month,DayofMonth,Origin,Dest,CRSDepTime,DepDelayMinutes,DepDel15,ArrTime,ArrDelayMinutes,CRSArrTime
1,1.0,7.0,17.0,83.0,2.0,1019.0,13.0,9.0,21.0,21.0,155.0,80.0,72.0,9.0,10.0,6.0,100.0,0.7,1005.0,16.0,3.0,6.0,4.0,63.0,97.0,72.0,11.0,2016.0,1.0,3.0,13.0,11.0,9.0,2105.0,27.0,1.0,24.0,0.0,27.0
2,1.0,13.0,7.0,34.0,0.1,1008.0,19.0,10.0,28.0,28.0,140.0,29.0,233.0,2.0,19.0,13.0,0.0,0.0,1012.0,24.0,10.0,21.0,21.0,274.0,55.0,233.0,11.0,2016.0,3.0,8.0,21.0,11.0,2.0,1720.0,64.0,1.0,2357.0,0.0,1.0
5,1.0,5.0,23.0,67.0,0.0,1014.0,8.0,3.0,24.0,24.0,118.0,96.0,161.0,5.0,10.0,17.0,75.0,0.1,1012.0,11.0,10.0,20.0,19.0,210.0,85.0,161.0,8.0,2016.0,2.0,6.0,10.0,8.0,5.0,1855.0,15.0,1.0,122.0,22.0,100.0
13,1.0,12.0,22.0,0.0,0.0,1016.0,15.0,10.0,31.0,31.0,224.0,59.0,576.0,8.0,8.0,17.0,0.0,0.0,1017.0,10.0,10.0,28.0,28.0,246.0,52.0,576.0,14.0,2017.0,3.0,7.0,30.0,14.0,8.0,1145.0,35.0,1.0,717.0,0.0,728.0
16,1.0,14.0,15.0,79.0,0.1,1015.0,16.0,6.0,20.0,18.0,237.0,83.0,133.0,8.0,17.0,-2.0,0.0,0.0,1011.0,20.0,10.0,34.0,33.0,160.0,11.0,133.0,7.0,2016.0,2.0,5.0,13.0,7.0,8.0,1509.0,78.0,1.0,853.0,6.0,847.0


In [0]:
pred_df.to_csv("pred.csv")
!cp pred.csv "gdrive/My Drive/Colab Notebooks/"